In [1]:
import pandas as pd
from util import OpenAIGPT
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
igpt = OpenAIGPT(keys_path="gpt3keys.txt")

In [2]:
igpt('hello')

'Hi there! How can I assist you today?'

In [23]:
from sklearn.model_selection import train_test_split
data = pd.read_csv('../data/travel_insurance/travel_insurance.csv')
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print("Train set shape:", train_data.shape)
print("Test set shape:", test_data.shape)

Train set shape: (1589, 9)
Test set shape: (398, 9)


### Process data with multi thread


In [24]:
def process_row(row):
    try:
        row_copy = row.drop(labels=['TravelInsurance'])
        prompt = f'''
        Write a brief sentence summarizing these details into a coherent profile description.
        {row_copy}
        '''
        res = igpt(prompt)

        label = 'A. negative' if row['TravelInsurance'] == 0 else 'B. positive'

        return {
            "instruction": "Determine whether the person will buy Travel insurance, you will output: {A. positive / B. negative}",
            "input": res,
            "output": label,
            "history": []
        }
    except Exception as e:
        print(f"Error processing row: {e}")
        return None


def generate_dataset(df, max_samples=1000):
    global igpt
    igpt = OpenAIGPT(model_name='gpt-3.5-turbo', keys_path='gpt3keys.txt')
    df_subset = df.head(max_samples)
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(process_row, row) for _, row in df_subset.iterrows()]
        for future in tqdm(futures, total=len(futures), desc="Processing rows"):
            result = future.result()
            if result:
                results.append(result)

    return results

In [ ]:
# dataset = generate_dataset(train_data)
# output_file_path = '../data/travel_insurance/travel_insurance.json'
# with open(output_file_path, 'w', encoding='utf-8') as f:
#     json.dump(dataset, f, ensure_ascii=False, indent=2)

In [25]:
# dataset = generate_dataset(test_data)
# output_file_path = '../data/travel_insurance/travel_insurance_test.json'
# with open(output_file_path, 'w', encoding='utf-8') as f:
#     json.dump(dataset, f, ensure_ascii=False, indent=2)

### Create more label


In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from util import OpenAIGPT
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

igpt = OpenAIGPT(keys_path="gpt3keys.txt")
data = pd.read_csv('../data/travel_insurance/travel_insurance.csv')
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print("Train set shape:", train_data.shape)
print("Test set shape:", test_data.shape)

'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Train set shape: (1589, 9)
Test set shape: (398, 9)


/Users/yuzheyang/anaconda3/envs/torch_env/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
def match_choice(text):
    match = re.findall(r'.*?([A-D]+(?:[、, ]+[A-D]+)*)', text)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ]+', last_match))
    return ''


def process_row(row):
    try:
        row_copy = row.drop(labels=['TravelInsurance'])
        prompt = f'''
        Write a brief sentence summarizing these details into a coherent profile description.
        {row_copy}
        '''
        res = igpt(prompt)

        label = ''

        # row: Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
        if row['TravelInsurance'] == 0:
            prompt_label = f'''
            This is the profile of a person who will not buy Travel insurance: {res}
            Please select the most appropriate option to judge the willingness to buy the travel insurance: (A. very negative / B. negative). You do not need to provide a reason.
            '''
            label = igpt(prompt_label)

        if row['TravelInsurance'] == 1:
            prompt_label = f'''
            This is the profile of a person who will buy Travel insurance: {res}
            Please select the most appropriate option to judge the willingness to buy the travel insurance: (C. positive / D. very positive). You do not need to provide a reason.
            '''
            label = igpt(prompt_label)

        return {
            "instruction": "Determine whether the person will buy Travel insurance, you will output: (A. very negative / B. negative / C. positive / D. very positive)",
            "input": res,
            "output": label,
            "history": []
        }
    except Exception as e:
        print(f"Error processing row: {e}")
        return None


def generate_dataset(df, max_samples=1000):
    global igpt
    igpt = OpenAIGPT(model_name='gpt-3.5-turbo', keys_path='gpt3keys.txt')
    df_subset = df.head(max_samples)
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(process_row, row) for _, row in df_subset.iterrows()]
        for future in tqdm(futures, total=len(futures), desc="Processing rows"):
            result = future.result()
            if result:
                results.append(result)

    return results

### test data


In [10]:
dataset = generate_dataset(test_data, max_samples=100)
output_file_path = '../data/travel_insurance_4_label/test.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=2)

Processing rows:   0%|          | 0/100 [00:00<?, ?it/s]

Processing rows: 100%|██████████| 100/100 [01:23<00:00,  1.20it/s]


### train data


In [4]:
dataset = generate_dataset(train_data, max_samples=1000)
output_file_path = '../data/travel_insurance_4_label/train.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=2)

Processing rows: 100%|██████████| 1000/1000 [14:53<00:00,  1.12it/s]
